# Class 2: Time Series Analysis with Statistical Modeling

This notebook delves into statistical models commonly used for time series analysis and forecasting. We will cover AR, MA, ARMA, ARIMA, SARIMAX, and GARCH models, along with model selection criteria and diagnostic checks.

## 0. Setup and Imports

In [ ]:
# Class 2: Time Series Analysis with Statistical Modeling - Python Demonstrations

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pmdarima as pm
from arch import arch_model
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
import warnings

warnings.filterwarnings("ignore") # Ignore harmless warnings

## 1. Load Data and Prepare

*Concept placeholder* 
 (No specific concept outline found for this section)

In [ ]:
print("--- 1. Loading Data ---")
data_file = "/home/ubuntu/aapl_stock_data_10y.csv"
df = pd.read_csv(data_file, index_col='Date', parse_dates=True)

# Use Adjusted Close price
ts = df['Adj Close'].copy()
# Use Volume as an exogenous variable example
exog = df['Volume'].copy()

# Use log returns for GARCH modeling (common practice)
log_returns = np.log(ts / ts.shift(1)).dropna()

# Split data: Train (first 9 years), Test (last 1 year approx)
# ~252 trading days per year
train_size = len(ts) - 252
train_ts, test_ts = ts[:train_size], ts[train_size:]
train_exog, test_exog = exog[:train_size], exog[train_size:]
train_log_returns, test_log_returns = log_returns[:train_size], log_returns[train_size:]

print(f"Train set size: {len(train_ts)}")
print(f"Test set size: {len(test_ts)}")

# Initialize variables for metrics to avoid NameError if a model fails
arima_rmse, arima_mae = np.nan, np.nan
auto_arima_rmse, auto_arima_mae = np.nan, np.nan
sarimax_rmse, sarimax_mae = np.nan, np.nan
best_order = (0,0,0) # Default order

**Interpretation:**

*Interpretation placeholder* 
 (No specific interpretation found for this section)

## 2. ARIMA Model

*Concept placeholder* 
 (No specific concept outline found for this section)

In [ ]:
print("\n--- 2. Fitting ARIMA Model ---")
# Based on Class 1, the series needed differencing (d=1).
# ACF/PACF of differenced series can suggest p, q. Let's try ARIMA(1,1,1) as a starting point.
# Note: Order selection is iterative. ACF/PACF gives hints.

try:
    arima_model = ARIMA(train_ts, order=(1, 1, 1))
    arima_fit = arima_model.fit()
    print(arima_fit.summary())

    # Forecast
    arima_pred = arima_fit.predict(start=len(train_ts), end=len(ts)-1)

    # Plot forecast vs actual
    plt.figure(figsize=(12, 6))
    plt.plot(train_ts.index, train_ts, label='Train')
    plt.plot(test_ts.index, test_ts, label='Test')
    plt.plot(test_ts.index, arima_pred, label='ARIMA(1,1,1) Forecast')
    plt.title('ARIMA(1,1,1) Forecast vs Actuals')
    plt.xlabel('Date')
    plt.ylabel('Price (USD)')
    plt.legend()
    plt.grid(True)
    plt.savefig("/home/ubuntu/plot_09_arima_forecast.png")
    plt.close()
    print("Saved plot: plot_09_arima_forecast.png")

    # Performance Metrics
    arima_rmse = np.sqrt(mean_squared_error(test_ts, arima_pred))
    arima_mae = mean_absolute_error(test_ts, arima_pred)
    print(f"ARIMA(1,1,1) RMSE: {arima_rmse:.4f}")
    print(f"ARIMA(1,1,1) MAE: {arima_mae:.4f}")

except Exception as e:
    print(f"Error fitting ARIMA(1,1,1): {e}")

**Interpretation:**

*   **ARIMA(1,1,1) Summary:**
    *   The summary showed significant coefficients for both the AR1 (autoregressive term at lag 1) and MA1 (moving average term at lag 1) components (p-values << 0.05). This supports the inclusion of these terms in the model based on the training data.
    *   The `sigma2` value represents the estimated variance of the residuals.
*   **Auto ARIMA Summary:**
    *   The `auto_arima` trace showed the stepwise search process, comparing different model orders based on AIC. It converged to ARIMA(1,1,2) as the best model for the training data (AIC: 9225.391).
    *   The summary for the selected ARIMA(1,1,2) model showed significant coefficients for AR1, MA1, and MA2 terms.
*   **SARIMAX(1,1,1) Summary:**
    *   The coefficient for the exogenous variable `x1` (Volume) was statistically significant (p=0.005). This indicates that, within the training data, volume had a statistically significant relationship with the differenced adjusted close price, even though it didn't improve the test set forecast accuracy in this instance. The coefficient was negative, suggesting a slight inverse relationship in this specific model context.
    *   AR1 and MA1 terms were also highly significant, similar to the ARIMA model.
    *   A warning about the covariance matrix being singular or near-singular suggests potential multicollinearity or numerical instability issues, which might warrant further investigation or model simplification.
*   **GARCH(1,1) Summary (on Log Returns):**
    *   The model was fitted to the log returns to analyze volatility.
    *   The `mu` (mean) coefficient was significant but small, indicating a slight positive drift in daily log returns.
    *   The volatility parameters `alpha[1]` (ARCH term) and `beta[1]` (GARCH term) were both highly significant (p-values << 0.05).
    *   The sum of `alpha[1]` (0.10) and `beta[1]` (0.88) is close to 1 (0.98). This indicates high persistence in volatility – meaning that periods of high volatility tend to be followed by more high volatility, and periods of low volatility by low volatility (volatility clustering), a well-known characteristic of financial time series.
    *   The plot `plot_12_garch_volatility.png` visually confirms this, showing periods where the estimated conditional volatility spikes and remains elevated.

## 3. AUTO ARIMA

*Concept placeholder* 
 (No specific concept outline found for this section)

In [ ]:
print("\n--- 3. Fitting AUTO ARIMA Model ---")
# Automatically find best ARIMA model
try:
    auto_arima_model = pm.auto_arima(train_ts, 
                                     start_p=1, start_q=1,
                                     test='adf', # use adf test to find optimal 'd'
                                     max_p=3, max_q=3, # maximum p and q
                                     m=1, # Non-seasonal
                                     d=None, # let model determine 'd'
                                     seasonal=False, # No Seasonality
                                     start_P=0, D=0, 
                                     trace=True,
                                     error_action='ignore', 
                                     suppress_warnings=True, 
                                     stepwise=True) # Use stepwise algorithm

    print(auto_arima_model.summary())
    best_order = auto_arima_model.order # Store the best order found

    # Forecast with Auto ARIMA
    auto_arima_pred = auto_arima_model.predict(n_periods=len(test_ts))
    auto_arima_pred = pd.Series(auto_arima_pred, index=test_ts.index)

    # Plot forecast vs actual
    plt.figure(figsize=(12, 6))
    plt.plot(train_ts.index, train_ts, label='Train')
    plt.plot(test_ts.index, test_ts, label='Test')
    plt.plot(test_ts.index, auto_arima_pred, label='Auto ARIMA Forecast')
    plt.title('Auto ARIMA Forecast vs Actuals')
    plt.xlabel('Date')
    plt.ylabel('Price (USD)')
    plt.legend()
    plt.grid(True)
    plt.savefig("/home/ubuntu/plot_10_auto_arima_forecast.png")
    plt.close()
    print("Saved plot: plot_10_auto_arima_forecast.png")

    # Performance Metrics
    auto_arima_rmse = np.sqrt(mean_squared_error(test_ts, auto_arima_pred))
    auto_arima_mae = mean_absolute_error(test_ts, auto_arima_pred)
    print(f"Auto ARIMA RMSE: {auto_arima_rmse:.4f}")
    print(f"Auto ARIMA MAE: {auto_arima_mae:.4f}")

except Exception as e:
    print(f"Error fitting Auto ARIMA: {e}")
    # Use default order if auto_arima fails
    best_order = (1, 1, 1) # Fallback if auto arima fails
    print(f"Falling back to order {best_order} for SARIMAX due to Auto ARIMA error.")

**Interpretation:**

*   **ARIMA(1,1,1) Summary:**
    *   The summary showed significant coefficients for both the AR1 (autoregressive term at lag 1) and MA1 (moving average term at lag 1) components (p-values << 0.05). This supports the inclusion of these terms in the model based on the training data.
    *   The `sigma2` value represents the estimated variance of the residuals.
*   **Auto ARIMA Summary:**
    *   The `auto_arima` trace showed the stepwise search process, comparing different model orders based on AIC. It converged to ARIMA(1,1,2) as the best model for the training data (AIC: 9225.391).
    *   The summary for the selected ARIMA(1,1,2) model showed significant coefficients for AR1, MA1, and MA2 terms.
*   **SARIMAX(1,1,1) Summary:**
    *   The coefficient for the exogenous variable `x1` (Volume) was statistically significant (p=0.005). This indicates that, within the training data, volume had a statistically significant relationship with the differenced adjusted close price, even though it didn't improve the test set forecast accuracy in this instance. The coefficient was negative, suggesting a slight inverse relationship in this specific model context.
    *   AR1 and MA1 terms were also highly significant, similar to the ARIMA model.
    *   A warning about the covariance matrix being singular or near-singular suggests potential multicollinearity or numerical instability issues, which might warrant further investigation or model simplification.
*   **GARCH(1,1) Summary (on Log Returns):**
    *   The model was fitted to the log returns to analyze volatility.
    *   The `mu` (mean) coefficient was significant but small, indicating a slight positive drift in daily log returns.
    *   The volatility parameters `alpha[1]` (ARCH term) and `beta[1]` (GARCH term) were both highly significant (p-values << 0.05).
    *   The sum of `alpha[1]` (0.10) and `beta[1]` (0.88) is close to 1 (0.98). This indicates high persistence in volatility – meaning that periods of high volatility tend to be followed by more high volatility, and periods of low volatility by low volatility (volatility clustering), a well-known characteristic of financial time series.
    *   The plot `plot_12_garch_volatility.png` visually confirms this, showing periods where the estimated conditional volatility spikes and remains elevated.

*Note: Auto ARIMA failed during the prediction phase in the demonstration code.*

## 4. SARIMAX Model (Example with Exogenous Variable)

*Concept placeholder* 
 (No specific concept outline found for this section)

In [ ]:
print("\n--- 4. Fitting SARIMAX Model (with Exogenous Variable) ---")
# Using orders from Auto ARIMA (or fallback), add Volume as exogenous variable
# Note: Seasonality (P,D,Q,m) is set to 0 here as auto_arima found none.

try:
    # Ensure exog variables are numpy arrays
    train_exog_np = train_exog.values.reshape(-1, 1)
    test_exog_np = test_exog.values.reshape(-1, 1)

    sarimax_model = SARIMAX(train_ts, 
                            exog=train_exog_np, 
                            order=best_order, 
                            seasonal_order=(0, 0, 0, 0), # No seasonality assumed here
                            enforce_stationarity=False, 
                            enforce_invertibility=False)
    sarimax_fit = sarimax_model.fit(disp=False)
    print(sarimax_fit.summary())

    # Forecast with exogenous variables
    sarimax_pred = sarimax_fit.predict(start=len(train_ts), end=len(ts)-1, exog=test_exog_np)

    # Plot forecast vs actual
    plt.figure(figsize=(12, 6))
    plt.plot(train_ts.index, train_ts, label='Train')
    plt.plot(test_ts.index, test_ts, label='Test')
    plt.plot(test_ts.index, sarimax_pred, label='SARIMAX Forecast')
    plt.title(f'SARIMAX{best_order} Forecast vs Actuals (Exog: Volume)')
    plt.xlabel('Date')
    plt.ylabel('Price (USD)')
    plt.legend()
    plt.grid(True)
    plt.savefig("/home/ubuntu/plot_11_sarimax_forecast.png")
    plt.close()
    print("Saved plot: plot_11_sarimax_forecast.png")

    # Performance Metrics
    sarimax_rmse = np.sqrt(mean_squared_error(test_ts, sarimax_pred))
    sarimax_mae = mean_absolute_error(test_ts, sarimax_pred)
    print(f"SARIMAX{best_order} RMSE: {sarimax_rmse:.4f}")
    print(f"SARIMAX{best_order} MAE: {sarimax_mae:.4f}")

except Exception as e:
    print(f"Error fitting SARIMAX: {e}")

**Interpretation:**

*   **ARIMA(1,1,1) Summary:**
    *   The summary showed significant coefficients for both the AR1 (autoregressive term at lag 1) and MA1 (moving average term at lag 1) components (p-values << 0.05). This supports the inclusion of these terms in the model based on the training data.
    *   The `sigma2` value represents the estimated variance of the residuals.
*   **Auto ARIMA Summary:**
    *   The `auto_arima` trace showed the stepwise search process, comparing different model orders based on AIC. It converged to ARIMA(1,1,2) as the best model for the training data (AIC: 9225.391).
    *   The summary for the selected ARIMA(1,1,2) model showed significant coefficients for AR1, MA1, and MA2 terms.
*   **SARIMAX(1,1,1) Summary:**
    *   The coefficient for the exogenous variable `x1` (Volume) was statistically significant (p=0.005). This indicates that, within the training data, volume had a statistically significant relationship with the differenced adjusted close price, even though it didn't improve the test set forecast accuracy in this instance. The coefficient was negative, suggesting a slight inverse relationship in this specific model context.
    *   AR1 and MA1 terms were also highly significant, similar to the ARIMA model.
    *   A warning about the covariance matrix being singular or near-singular suggests potential multicollinearity or numerical instability issues, which might warrant further investigation or model simplification.
*   **GARCH(1,1) Summary (on Log Returns):**
    *   The model was fitted to the log returns to analyze volatility.
    *   The `mu` (mean) coefficient was significant but small, indicating a slight positive drift in daily log returns.
    *   The volatility parameters `alpha[1]` (ARCH term) and `beta[1]` (GARCH term) were both highly significant (p-values << 0.05).
    *   The sum of `alpha[1]` (0.10) and `beta[1]` (0.88) is close to 1 (0.98). This indicates high persistence in volatility – meaning that periods of high volatility tend to be followed by more high volatility, and periods of low volatility by low volatility (volatility clustering), a well-known characteristic of financial time series.
    *   The plot `plot_12_garch_volatility.png` visually confirms this, showing periods where the estimated conditional volatility spikes and remains elevated.

## 5. ARCH/GARCH Model for Volatility

*Concept placeholder* 
 (No specific concept outline found for this section)

In [ ]:
print("\n--- 5. Fitting GARCH Model on Log Returns ---")
# Model volatility of log returns
# Common choice: GARCH(1,1)
try:
    garch_model = arch_model(train_log_returns, vol='Garch', p=1, q=1)
    garch_fit = garch_model.fit(disp='off') # Turn off verbose fitting output
    print(garch_fit.summary())

    # Plot conditional volatility
    plt.figure(figsize=(12, 6))
    plt.plot(garch_fit.conditional_volatility, label='Conditional Volatility')
    plt.title('GARCH(1,1) Conditional Volatility of AAPL Log Returns')
    plt.xlabel('Date')
    plt.ylabel('Volatility')
    plt.legend()
    plt.grid(True)
    plt.savefig("/home/ubuntu/plot_12_garch_volatility.png")
    plt.close()
    print("Saved plot: plot_12_garch_volatility.png")
except Exception as e:
    print(f"Error fitting GARCH: {e}")

**Interpretation:**

*   **ARIMA(1,1,1) Summary:**
    *   The summary showed significant coefficients for both the AR1 (autoregressive term at lag 1) and MA1 (moving average term at lag 1) components (p-values << 0.05). This supports the inclusion of these terms in the model based on the training data.
    *   The `sigma2` value represents the estimated variance of the residuals.
*   **Auto ARIMA Summary:**
    *   The `auto_arima` trace showed the stepwise search process, comparing different model orders based on AIC. It converged to ARIMA(1,1,2) as the best model for the training data (AIC: 9225.391).
    *   The summary for the selected ARIMA(1,1,2) model showed significant coefficients for AR1, MA1, and MA2 terms.
*   **SARIMAX(1,1,1) Summary:**
    *   The coefficient for the exogenous variable `x1` (Volume) was statistically significant (p=0.005). This indicates that, within the training data, volume had a statistically significant relationship with the differenced adjusted close price, even though it didn't improve the test set forecast accuracy in this instance. The coefficient was negative, suggesting a slight inverse relationship in this specific model context.
    *   AR1 and MA1 terms were also highly significant, similar to the ARIMA model.
    *   A warning about the covariance matrix being singular or near-singular suggests potential multicollinearity or numerical instability issues, which might warrant further investigation or model simplification.
*   **GARCH(1,1) Summary (on Log Returns):**
    *   The model was fitted to the log returns to analyze volatility.
    *   The `mu` (mean) coefficient was significant but small, indicating a slight positive drift in daily log returns.
    *   The volatility parameters `alpha[1]` (ARCH term) and `beta[1]` (GARCH term) were both highly significant (p-values << 0.05).
    *   The sum of `alpha[1]` (0.10) and `beta[1]` (0.88) is close to 1 (0.98). This indicates high persistence in volatility – meaning that periods of high volatility tend to be followed by more high volatility, and periods of low volatility by low volatility (volatility clustering), a well-known characteristic of financial time series.
    *   The plot `plot_12_garch_volatility.png` visually confirms this, showing periods where the estimated conditional volatility spikes and remains elevated.

## 6. Model Diagnostics (Example: Auto ARIMA)

*Concept placeholder* 
 (No specific concept outline found for this section)

In [ ]:
print("\n--- 6. Diagnostic Checks (Example: Auto ARIMA) ---")
# Check if auto_arima_model exists and has residuals
if 'auto_arima_model' in locals() and hasattr(auto_arima_model, 'resid'):
    residuals = auto_arima_model.resid()
    
    # Plot Residuals
    plt.figure(figsize=(12, 4))
    plt.plot(residuals)
    plt.title('Auto ARIMA Residuals')
    plt.xlabel('Date')
    plt.ylabel('Residual Value')
    plt.grid(True)
    plt.savefig("/home/ubuntu/plot_13_auto_arima_residuals.png")
    plt.close()
    print("Saved plot: plot_13_auto_arima_residuals.png")

    # ACF/PACF of Residuals
    fig, axes = plt.subplots(1, 2, figsize=(16, 4))
    plot_acf(residuals, ax=axes[0], lags=40, title='ACF of Residuals')
    plot_pacf(residuals, ax=axes[1], lags=40, title='PACF of Residuals')
    plt.tight_layout()
    plt.savefig("/home/ubuntu/plot_14_residual_acf_pacf.png")
    plt.close()
    print("Saved plot: plot_14_residual_acf_pacf.png")
    print("Ideally, ACF/PACF of residuals should show no significant spikes.")

    # Ljung-Box Test
    try:
        ljung_box_result = acorr_ljungbox(residuals, lags=[20], return_df=True)
        print("\nLjung-Box Test on Residuals:")
        print(ljung_box_result)
        print("If p-value > 0.05, we fail to reject H0 (residuals are independent/white noise).")
    except Exception as e:
        print(f"Could not perform Ljung-Box test: {e}")
else:
    print("Skipping Auto ARIMA diagnostics as the model did not fit successfully.")

**Interpretation:**

We examined the residuals from the fitted Auto ARIMA model (ARIMA(1,1,2)) to assess its adequacy on the training data.

*   **Residual Plot (`plot_13_auto_arima_residuals.png`):** The residuals appear somewhat randomly distributed around zero, without obvious trends or strong patterns, which is desirable.
*   **ACF/PACF Plots (`plot_14_residual_acf_pacf.png`):** Ideally, the ACF and PACF plots of the residuals should show no significant spikes outside the confidence bands for all lags greater than 0. While many spikes are within the bands, there might be a few borderline or slightly significant spikes, suggesting some autocorrelation might remain.
*   **Ljung-Box Test:** This test formally checks if the residuals are independently distributed (i.e., resemble white noise). The null hypothesis (H0) is that the residuals are independent.
    *   The test yielded a p-value of approximately 0.011 for lags up to 20.
    *   Since the p-value (0.011) is less than the common significance level of 0.05, we **reject the null hypothesis**. This indicates that there is significant autocorrelation remaining in the residuals, and they do not behave like white noise.

**Interpretation:** The diagnostic checks, particularly the Ljung-Box test, suggest that the Auto ARIMA(1,1,2) model, while being the best fit according to AIC on the training data, did not fully capture all the temporal dependencies. There is still structure left in the residuals. This could mean a more complex model (perhaps higher orders, different differencing, or incorporating seasonality if relevant) might be needed, or that the underlying process has complexities not easily captured by standard ARIMA models.

## 7. Model Comparison (Metrics)

*Concept placeholder* 
 (No specific concept outline found for this section)

In [ ]:
print("\n--- 7. Model Performance Comparison (Test Set) ---")
print(f"ARIMA(1,1,1)   RMSE: {arima_rmse:.4f}, MAE: {arima_mae:.4f}")
print(f"Auto ARIMA     RMSE: {auto_arima_rmse:.4f}, MAE: {auto_arima_mae:.4f}")
print(f"SARIMAX{best_order}    RMSE: {sarimax_rmse:.4f}, MAE: {sarimax_mae:.4f}")
print("Note: Lower RMSE/MAE indicates better forecast accuracy on the test set.")

print("\nClass 2 Demonstrations Complete.")

**Interpretation:**

The primary goal of these models was to forecast the adjusted close price for the last year of the data (test set). We evaluated the forecasts using Root Mean Squared Error (RMSE) and Mean Absolute Error (MAE). Lower values indicate better accuracy.

*   **ARIMA(1,1,1):**
    *   RMSE: 35.0100
    *   MAE: 31.5656
*   **Auto ARIMA:**
    *   The `pmdarima.auto_arima` function successfully identified an optimal order (1,1,2) based on the AIC criterion during its search process. However, it encountered an error ("Input contains NaN") during the prediction phase on the test set. This resulted in NaN values for RMSE and MAE. This failure often indicates issues with how the model handles data transformations (like differencing) at the prediction boundary or potential NaNs introduced during feature engineering (though none were explicitly added here). Further investigation would be needed in a real-world scenario (e.g., checking internal steps, trying different `pmdarima` versions or settings).
*   **SARIMAX(1,1,1) (with Volume as Exogenous Variable):**
    *   Since Auto ARIMA failed to provide a reliable forecast, we used the fallback order (1,1,1) for the SARIMAX model, incorporating the daily trading volume as an external predictor.
    *   RMSE: 35.2970
    *   MAE: 31.8694

**Interpretation:**
Comparing the successful models, the simple ARIMA(1,1,1) model performed slightly better than the SARIMAX(1,1,1) model with volume as an exogenous variable on this specific test set, having marginally lower RMSE and MAE. This suggests that, for this particular dataset and model configuration, adding volume did not improve the one-step-ahead forecast accuracy for the price itself. It is important to note that stock price forecasting is notoriously difficult, and these relatively high error values (compared to the price scale) are common. The value of exogenous variables might be more apparent in different contexts or with different modeling approaches.

## Overall Findings & Recommendations

*   **Process:** The demonstration successfully walked through loading, splitting, modeling (ARIMA, Auto ARIMA, SARIMAX, GARCH), forecasting, evaluating (RMSE, MAE), and diagnosing time series models.
*   **Stationarity:** The initial ADF test confirmed the non-stationarity of the raw price series, necessitating differencing (d=1) for ARIMA-based models.
*   **Model Selection:** Auto ARIMA is a useful tool for automatically finding potentially good ARIMA orders based on information criteria (AIC/BIC), but it's not foolproof (as seen by the prediction failure and residual issues). Manual inspection of ACF/PACF plots remains important.
*   **Exogenous Variables:** SARIMAX allows incorporating external factors. While Volume was statistically significant in the training fit, it didn't improve test set price forecasts here. The impact of exogenous variables can be highly context-dependent.
*   **Volatility Modeling:** GARCH models are essential for analyzing and forecasting the *volatility* (risk) of financial returns, capturing the common phenomenon of volatility clustering.
*   **Diagnostics:** Residual analysis is crucial. Even if a model looks good on paper (e.g., low AIC) or performs reasonably on forecasts, checking residuals (plots, Ljung-Box test) reveals if the model assumptions are met and if predictable patterns remain.
The failure of the Auto ARIMA residuals to pass the Ljung-Box test is a key teaching point about the limitations of models and the importance of diagnostics.
*   **Forecasting Difficulty:** Emphasize that stock price forecasting is inherently challenging due to market efficiency and noise. Simple models often perform similarly to more complex ones for point forecasts.
*   **Next Steps (in a real project):** Investigate the Auto ARIMA prediction failure. Explore higher-order models or different structures based on residual diagnostics. Consider non-linear models or Machine Learning approaches (Class 3).